In [1]:
library(tidyverse)
install.packages("lubridate")  
library(lubridate)
library(dplyr)
library(readr)
library(ggplot2)

players <- read_csv("/home/jovyan/work/dsci-100-student/project/players.csv")
sessions <- read_csv("/home/jovyan/work/dsci-100-student/project/sessions.csv")

merged_data <- left_join(players,sessions, by = "hashedEmail")

merged_data <- merged_data|>
    select(-hashedEmail,-original_start_time,-original_end_time)
    merged_data$Age[is.na(merged_data$Age)] <- mean(merged_data$Age, na.rm = TRUE)

merged_data


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Colum

experience,subscribe,played_hours,name,gender,Age,start_time,end_time
<chr>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
Pro,TRUE,30.3,Morgan,Male,9,08/08/2024 00:21,08/08/2024 01:35
Pro,TRUE,30.3,Morgan,Male,9,09/09/2024 22:30,09/09/2024 22:37
Pro,TRUE,30.3,Morgan,Male,9,08/08/2024 02:41,08/08/2024 03:25
Pro,TRUE,30.3,Morgan,Male,9,10/09/2024 15:07,10/09/2024 15:29
Pro,TRUE,30.3,Morgan,Male,9,05/05/2024 22:21,05/05/2024 23:17
Pro,TRUE,30.3,Morgan,Male,9,06/04/2024 22:24,06/04/2024 23:33
Pro,TRUE,30.3,Morgan,Male,9,20/04/2024 20:46,20/04/2024 21:48
Pro,TRUE,30.3,Morgan,Male,9,15/06/2024 16:37,15/06/2024 18:37
Pro,TRUE,30.3,Morgan,Male,9,05/05/2024 23:40,06/05/2024 00:55


To get a large amount of data, total sessions, total play time, and average session duration are key indicators.  
Let’s call these **engagement level metrics**.

1. Relation between "age" and engagement level metrics  
    > Do older players spend more time per session? (age vs average session duration)

2. Relation between "experience" and engagement level metrics  
    > Do players with more experience tend to play more? (experience vs Avergae Played Hours)

3. Relation between "gender" and engagement level metrics  
    > Which gender has more sessions? (gender vs total sessions)

4. Relation between "subscription" and engagement level metrics  
    > Do subscribers tend to play more than non-subscribers? (subscribe vs total play time)


**1. Do older players spend more time per session? (age vs average session duration)**

In [3]:
merged_data <- merged_data|>
    mutate(
        start_time = as.POSIXct(start_time, format = "%m/%d/%Y %H:%M"),
        end_time = as.POSIXct(end_time, format = "%m/%d/%Y %H:%M"),
        session_duration = as.numeric(difftime(end_time, start_time, units = "mins"))
    )

# Outliers were detected.(start_time,end time)
# Checking sum of all session time and duplicated session time are same.
name_order <- merged_data |>
  count(name, sort = TRUE) |>
  pull(name)

merged_data$name <- factor(merged_data$name, levels = name_order)

merged_data <- merged_data |>
  arrange(name,start_time)

# merged_data <- merged_data |>
#     filter(name == "Morgan") |>
#     arrange(start_time)
# The outliers represented non-overlapping, extremely long sessions and were unlikely to affect the analysis, so we removed them from the dataset.

merged_data_clean <- merged_data |>
  filter(session_duration <= 1440 ,!is.na(start_time),!is.na(end_time))

merged_data_clean


experience,subscribe,played_hours,name,gender,Age,start_time,end_time,session_duration
<chr>,<lgl>,<dbl>,<fct>,<chr>,<dbl>,<dttm>,<dttm>,<dbl>
Amateur,TRUE,56.1,Dana,Male,23,2024-01-06 02:22:00,2024-01-06 03:37:00,75
Amateur,TRUE,56.1,Dana,Male,23,2024-01-06 05:45:00,2024-01-06 06:57:00,72
Amateur,TRUE,56.1,Dana,Male,23,2024-01-06 17:01:00,2024-01-06 18:46:00,105
Amateur,TRUE,56.1,Dana,Male,23,2024-01-06 20:01:00,2024-01-06 21:32:00,91
Amateur,TRUE,56.1,Dana,Male,23,2024-01-06 23:51:00,2024-01-06 23:55:00,4
Amateur,TRUE,56.1,Dana,Male,23,2024-01-07 23:07:00,2024-01-07 23:14:00,7
Amateur,TRUE,56.1,Dana,Male,23,2024-01-08 21:28:00,2024-01-08 21:33:00,5
Amateur,TRUE,56.1,Dana,Male,23,2024-01-08 21:34:00,2024-01-08 21:39:00,5
Amateur,TRUE,56.1,Dana,Male,23,2024-01-08 21:40:00,2024-01-08 21:44:00,4


**3. Which gender has more sessions? (gender vs total sessions)**

In [1]:
gender_sessions <- sessions |>
filter(!is.na(gender)) |>
select(name, gender, start_time, end_time, session_duration) |>
group_by(gender) |>
summarize(total_sessions=n()) 
gender_sessions

gender_sessions_plot <- gender_sessions |>
ggplot(aes(x=gender, y=total_sessions, fill=gender)) +
geom_bar(stat="identity") +
labs(title="Total Sessions Related to Gender", x="Player's Gender", y="Total Count") +
theme(text = element_text(size=20))
gender_sessions_plot

ERROR: Error in summarize(group_by(select(filter(sessions, !is.na(gender)), name, : could not find function "summarize"


1. Filter out rows with missing gender(NA) from gender column using "filter" function.
2. Select five column names (name, gender, start_time, end_time, session_duration) from data using *select* function.
3. Group the data by gender using *group_by* function.
4. Calculate total number of rows in sessions data for each gender using *summarize* function, then make a new name: total_sessions.
5. Print out gender_sessions.
6. Create a plot for relationship between gender and total sessions using gender_sessions data.
7. Using x = gender, y = total_sessions and fill gender to create a bar chart and create a title and each name of xlab and ylab using *labs* to combine it.